In [40]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv("sampled_x_train.csv")  # Randomized sample of the big dataset

/tmp/ipykernel_8269/2234297548.py:1: DtypeWarning: Columns (107,109,110,114,116,117,121,123,124,129,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("sampled_x_train.csv")  # Randomized sample of the big dataset


In [42]:
def cleaning(
    df: pd.DataFrame,
    piezo: bool = False,
    meteo: bool = False,
    hydro: bool = False,
    prelev: bool = False,
    insee: bool = False
) -> pd.DataFrame:
    # Define columns to exclude for each category
    excluded_piezo = [
        "piezo_station_department_code",
        "piezo_station_update_date",
        "piezo_station_department_name",
        "piezo_station_commune_code_insee",
        "piezo_station_pe_label",
        "piezo_station_bdlisa_codes",
        "piezo_station_bss_code",
        "piezo_station_commune_name",
        "piezo_station_bss_id",
        "piezo_bss_code",
        "piezo_continuity_name",
        "piezo_producer_code",
        "piezo_producer_name",
        "piezo_measure_nature_name",
    ]
    excluded_meteo = ["meteo_id", "meteo_radiation_IR", "meteo_name"]
    excluded_hydro = [
        "hydro_station_code",
        "hydro_status_label",
        "hydro_method_code",
        "hydro_method_label",
        "hydro_qualification_label",
    ]
    excluded_prelev = [
        "prelev_structure_code_0",
        "prelev_volume_0",
        "prelev_usage_label_0",
        "prelev_volume_obtention_mode_label_0",
        "prelev_longitude_0",
        "prelev_latitude_0",
        "prelev_commune_code_insee_0",
        "prelev_structure_code_1",
        "prelev_volume_1",
        "prelev_usage_label_1",
        "prelev_volume_obtention_mode_label_1",
        "prelev_longitude_1",
        "prelev_latitude_1",
        "prelev_commune_code_insee_1",
        "prelev_structure_code_2",
        "prelev_volume_2",
        "prelev_usage_label_2",
        "prelev_volume_obtention_mode_label_2",
        "prelev_longitude_2",
        "prelev_latitude_2",
        "prelev_commune_code_insee_2",
        "prelev_other_volume_sum",
    ]
    excluded_insee = ['prelev_commune_code_insee_0', 'prelev_commune_code_insee_1',
       'prelev_commune_code_insee_2', 'insee_%_agri', 'insee_pop_commune',
       'insee_med_living_level', 'insee_%_ind', 'insee_%_const']
    # Combine columns to drop based on the parameters
    columns_to_drop = []
    if piezo:
        columns_to_drop += excluded_piezo
    if meteo:
        columns_to_drop += excluded_meteo
    if hydro:
        columns_to_drop += excluded_hydro
    if prelev:
        columns_to_drop += excluded_prelev
    if insee:
        columns_to_drop += excluded_insee

    # Drop columns safely (ignore errors for missing columns)

    df_filtered = df.drop(columns=columns_to_drop, errors="ignore")

    return df_filtered

# Cleaning all redundant value

In [43]:
# Removing redundant columns
df_filtered = cleaning(df, True, True, True, True, True)
df_filtered.head(2)

,piezo_station_investigation_depth,piezo_station_altitude,piezo_station_longitude,piezo_station_latitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,piezo_measure_nature_code,...,distance_piezo_meteo,hydro_observation_date_elab,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,hydro_longitude,hydro_latitude,hydro_hydro_quantity_elab,distance_piezo_hydro,piezo_groundwater_level_category
0,8.0,215.00,4.066361,49.953581,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,...,3.595843,2020-01-01,7603.0,16,20,4.078194,49.924151,QmJ,3.380271,Very High
1,12.9,215.66,3.358034,46.484443,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,...,7.624683,2020-01-01,234640.0,16,20,3.325951,46.561117,QmJ,8.872130,Low


# Unifies longitude and latitude columns

In [44]:
def unify_long_lat(df: pd.DataFrame, distance_threshold: int = 25) -> pd.DataFrame:
    """
    Unifies longitude and latitude into single columns if distance_piezo_hydro is below the threshold.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.
        distance_threshold (int): The maximum distance for unification.

    Returns:
        pd.DataFrame: The modified DataFrame with unified longitude and latitude columns.
    """
    # Columns to exclude after processing
    excluding = [
        "piezo_station_latitude",
        "piezo_station_longitude",
        "hydro_longitude",
        "hydro_latitude",
        "meteo_longitude",
        "meteo_latitude",
        "distance_piezo_hydro",
        "distance_hydro_meteo",
    ]

    # Ensure the required column exists
    if "distance_piezo_hydro" in df.columns:
        # Create unified longitude and latitude where condition is met
        df.loc[df["distance_piezo_hydro"] < distance_threshold, "longitude"] = df[
            "piezo_station_longitude"
        ]
        df.loc[df["distance_piezo_hydro"] < distance_threshold, "latitude"] = df[
            "piezo_station_latitude"
        ]

    # Drop the excluded columns
    df = df.drop(columns=excluding, errors="ignore")

    return df

In [45]:
df_filtered = unify_long_lat(df_filtered)

In [46]:
df_filtered.head()

,piezo_station_investigation_depth,piezo_station_altitude,piezo_measurement_date,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,piezo_measure_nature_code,meteo_altitude,meteo_date,...,meteo_snow_thickness_6h,distance_piezo_meteo,hydro_observation_date_elab,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,hydro_hydro_quantity_elab,piezo_groundwater_level_category,longitude,latitude
0,8.0,215.00,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,165,2020-01-01,...,NaN,3.595843,2020-01-01,7603.0,16,20,QmJ,Very High,4.066361,49.953581
1,12.9,215.66,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,275,2020-01-01,...,NaN,7.624683,2020-01-01,234640.0,16,20,QmJ,Low,3.358034,46.484443
2,13.0,248.07,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,250,2020-01-01,...,NaN,4.625349,2020-01-01,450.0,16,20,QmM,High,3.389036,46.261645
3,323.0,30.00,2020-01-01,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,32,2020-01-01,...,NaN,4.758941,2020-01-01,9608.0,12,20,QmJ,Very High,7.125901,43.640615
4,190.0,60.00,2020-01-01,Mode d'obtention inconnu,Donnée contrôlée niveau 1,Correcte,2.0,0,32,2020-01-01,...,NaN,9.680241,2020-01-01,9608.0,12,20,QmJ,Very High,7.113075,43.686213


In [47]:
df_filtered.select_dtypes(include=["object"]).columns

Index(['piezo_measurement_date', 'piezo_obtention_mode', 'piezo_status',
       'piezo_qualification', 'piezo_measure_nature_code', 'meteo_date',
       'hydro_observation_date_elab', 'hydro_hydro_quantity_elab',
       'piezo_groundwater_level_category'],
      dtype='object')

In [48]:
date_columns = df_filtered.filter(like="date", axis=1).columns
df_date = df_filtered[date_columns]
df_date.head()

,piezo_measurement_date,meteo_date,hydro_observation_date_elab
0,2020-01-01,2020-01-01,2020-01-01
1,2020-01-01,2020-01-01,2020-01-01
2,2020-01-01,2020-01-01,2020-01-01
3,2020-01-01,2020-01-01,2020-01-01
4,2020-01-01,2020-01-01,2020-01-01


In [49]:
def unify_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Unifies date combine date in a single column.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the data.

    Returns:
        pd.DataFrame: The modified DataFrame with unifies dates columns.
    """
    # Columns to exclude after processing
    excluding = ["piezo_measurement_date", "meteo_date", "hydro_observation_date_elab"]

    # Check and assign the first available date column
    if "piezo_measurement_date" in df.columns:
        df["date"] = df["piezo_measurement_date"]

    df = df.drop(columns=excluding, errors="ignore")

    return df

In [50]:
df_unify = unify_date(df_filtered)
df_unify.head()

,piezo_station_investigation_depth,piezo_station_altitude,piezo_obtention_mode,piezo_status,piezo_qualification,piezo_continuity_code,piezo_measure_nature_code,meteo_altitude,meteo_rain_height,meteo_DRR,...,meteo_snow_thickness_6h,distance_piezo_meteo,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,hydro_hydro_quantity_elab,piezo_groundwater_level_category,longitude,latitude,date
0,8.0,215.00,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,165,0.2,NaN,...,NaN,3.595843,7603.0,16,20,QmJ,Very High,4.066361,49.953581,2020-01-01
1,12.9,215.66,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,275,0.0,NaN,...,NaN,7.624683,234640.0,16,20,QmJ,Low,3.358034,46.484443,2020-01-01
2,13.0,248.07,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,0,250,0.2,NaN,...,NaN,4.625349,450.0,16,20,QmM,High,3.389036,46.261645,2020-01-01
3,323.0,30.00,Valeur mesurée,Donnée contrôlée niveau 2,Correcte,2.0,N,32,0.2,NaN,...,NaN,4.758941,9608.0,12,20,QmJ,Very High,7.125901,43.640615,2020-01-01
4,190.0,60.00,Mode d'obtention inconnu,Donnée contrôlée niveau 1,Correcte,2.0,0,32,0.2,NaN,...,NaN,9.680241,9608.0,12,20,QmJ,Very High,7.113075,43.686213,2020-01-01


In [55]:
from faiss_imputer import FaissImputer
from sklearn.impute import SimpleImputer
    
def impute_na(df: pd.DataFrame) -> pd.DataFrame:
    """
    Impute missing values in a DataFrame using FaissImputer.
    
    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame with missing values.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame with missing values imputed.
    """
    # Drop non-numerical columns
    categorical_cols = df.select_dtypes(include=['object', 'datetime64']).columns
    df = df.drop(columns=categorical_cols)
    
    # Create an instance of FaissImputer
    imputer = FaissImputer(n_neighbors=10)
    simple_imputer = SimpleImputer(strategy='median')

    # Fit the imputer on the data frame with missing values
    imputer.fit(df)

    # Transform the data frame with missing values
    df = pd.DataFrame(imputer.transform(df), columns=df.columns)
    df = pd.DataFrame(simple_imputer.fit_transform(df), columns=df.columns)
    
    return df

In [56]:
df_imputed = impute_na(df_unify)
df_imputed.head()

/home/nadia/miniconda3/envs/everything/lib/python3.11/site-packages/faiss_imputer/faiss_imputer.py:99: RuntimeWarning: Mean of empty slice
  column_agg = np.nanmean(selected_values, axis=0)


,piezo_station_investigation_depth,piezo_station_altitude,piezo_continuity_code,meteo_altitude,meteo_rain_height,meteo_DRR,meteo_temperature_min,meteo_time_tn,meteo_temperature_max,meteo_time_tx,...,meteo_radiation_UV,meteo_snow_height,meteo_snow_thickness_max,meteo_snow_thickness_6h,distance_piezo_meteo,hydro_observation_result_elab,hydro_status_code,hydro_qualification_code,longitude,latitude
0,8.0,215.000000,2.0,165.0,0.2,0.0,-1.6,2013.0,2.1,1304.0,...,0.8675,0.0,0.0,0.0,3.595843,7603.0,16.0,20.0,4.066361,49.953583
1,12.9,215.660004,2.0,275.0,0.0,0.0,-1.3,38.0,5.9,1457.0,...,0.8675,0.0,0.0,0.0,7.624683,234640.0,16.0,20.0,3.358034,46.484444
2,13.0,248.070007,2.0,250.0,0.2,0.0,-3.8,600.0,8.3,1426.0,...,0.8675,0.0,0.0,0.0,4.625349,450.0,16.0,20.0,3.389036,46.261646
3,323.0,30.000000,2.0,32.0,0.2,0.0,2.0,433.0,14.8,1221.0,...,0.8675,0.0,0.0,0.0,4.758941,9608.0,12.0,20.0,7.125901,43.640614
4,190.0,60.000000,2.0,32.0,0.2,0.0,2.0,433.0,14.8,1221.0,...,0.8675,0.0,0.0,0.0,9.680241,9608.0,12.0,20.0,7.113075,43.686214
